In [ ]:
import json
import demjson3
from tqdm import tqdm
import ast


In [ ]:
with open('../steam_games.json', 'r') as f: # name of game
    metadata = [eval(json.loads(demjson3.encode(line))) for line in f]


with open('../steam_new.json') as f: # interaction
    reviews = [eval(json.loads(demjson3.encode(line))) for line in f]

In [ ]:
metadata[0]

In [ ]:
reviews[0]

In [ ]:
users = set()
items = set()
for review in tqdm(reviews):
    users.add(review['username'])
    items.add(review['product_id'])

In [ ]:
item2id = dict()
count = 0
for item in items:
    item2id[item] = count
    count += 1
print(len(users), len(items), len(reviews), len(reviews) / (len(users) * len(items)))

In [ ]:
id_title = {}
id_item = {}
cnt = 0
for meta in tqdm(metadata):
    if 'title' in meta.keys() and 'id' in meta.keys() and len(meta['title']) > 1: # remove the item without title
        id_title[meta['id']] = meta['title']

In [ ]:
from collections import defaultdict
import numpy as np


user_num_dict = defaultdict(int)
item_num_dict = defaultdict(int)
for review in tqdm(reviews):
    user = review['username']
    item = review['product_id']

    user_num_dict[user] += 1
    item_num_dict[item] += 1


user_nocold = []
item_nocold = []
for user in user_num_dict.keys():
    if user_num_dict[user] >= 20:
        user_nocold.append(user)

for item in item_num_dict.keys():
    if item_num_dict[item] >= 20:
        item_nocold.append(item)

In [ ]:

print(len(user_nocold), len(item_nocold))

user_nocold = user_nocold[:int(len(user_nocold)*0.3)]
item_nocold = item_nocold[:int(len(user_nocold)*0.3)]

user_nocold = set(user_nocold)
item_nocold = set(item_nocold)

print(len(user_nocold), len(item_nocold))

In [ ]:
users = dict()
cc = 0
n_interaction = 0
for review in tqdm(reviews):
    user = review['username']
    if 'product_id' not in review:
        break
    item = review['product_id']
    if item not in id_title:
        continue
    if user not in user_nocold:
        continue
    if item not in item_nocold:
        continue
    if review['product_id'] not in id_item:
        id_item[review['product_id']] = cnt
        cnt += 1
    # if 'overall' not in review:
    #     continue
    if 'date' not in review:
        continue
    if user not in users:
        users[user] = {
            'items': [],
            'timestamps': [],
            'reviews': []
        }
    if item not in users[user]['items']:
        users[user]['items'].append(item)
        users[user]['timestamps'].append(review['date'])
        n_interaction += 1
    else:
        cc += 1


print(cc, n_interaction)       


In [ ]:
item2title_no_cold = {}
for meta in tqdm(metadata):
    if 'title' in meta.keys() and 'id' in meta.keys() and meta['id'] in item_nocold: # remove the item without title
        item2title_no_cold[meta['id']] = meta['title']

In [ ]:
print(item2title_no_cold)

In [ ]:
item2id_no_cold = {}
cnt = 0

for item in item2title_no_cold.keys():
    item2id_no_cold[item] = cnt
    cnt += 1

print(item2id_no_cold)
print(cnt)

In [ ]:
user_id = 0
interactions = []
B = []
for key in tqdm(users.keys()):
    items = users[key]['items']
    timestamps = users[key]['timestamps']
    all = list(zip(items, timestamps))
    res = sorted(all, key=lambda x: int(x[-1].replace("-", "")))
    items, timestamps = zip(*res)
    items, timestamps = list(items), list(timestamps)
    users[key]['items'] = items
    users[key]['item_ids'] = [item2id_no_cold[x] for x in items]
    users[key]['item_titles'] = [item2title_no_cold[x] for x in items]
    users[key]['timestamps'] = [t.replace("-", "") for t in timestamps]
    users[key]['timestamps_all'] = timestamps[0].replace("-", "")

In [ ]:
users_sorted = sorted(users.items(), key=lambda x: int(x[1]['timestamps_all']))
train_sessions = users_sorted[:int(len(users_sorted) * 0.8)]
val_sessions = users_sorted[int(len(users_sorted) * 0.8):int(len(users_sorted) * 0.9)]
test_sessions = users_sorted[int(len(users_sorted) * 0.9):]

In [ ]:
pad_item = len(item2id_no_cold.keys())

In [195]:
train_interactions = []

for kv in tqdm(train_sessions):
    k = kv[0]
    v = kv[1]
    if len(v['items']) > 10:
        for i in range(min(10, len(v['items']) - 1), len(v['items'])):
            st = max(i - 10, 0)
            train_interactions.append([k, v['items'][st: i], v['items'][i], v['item_ids'][st: i], v['item_ids'][i], v['item_titles'][st: i], v['item_titles'][i], int(v['timestamps'][i])])
    else:
        temp = [pad_item] * (10-len(v['items']))
        itemlist = v['item_ids'][:]
        itemlist.extend(temp)
        train_interactions.append([k, v['items'][:], v['items'][-1], itemlist, v['item_ids'][-1], v['item_titles'][:], v['item_titles'][-1], int(v['timestamps'][-1])])




100%|██████████| 9550/9550 [00:00<00:00, 37636.80it/s]


In [196]:
val_interactions = []

for kv in tqdm(val_sessions):
    k = kv[0]
    v = kv[1]
    if len(v['items']) > 10:
        for i in range(min(10, len(v['items']) - 1), len(v['items'])):
            st = max(i - 10, 0)
            val_interactions.append([k, v['items'][st: i], v['items'][i], v['item_ids'][st: i], v['item_ids'][i], v['item_titles'][st: i], v['item_titles'][i], int(v['timestamps'][i])])
    else:
        temp = [pad_item] * (10-len(v['items']))
        itemlist = v['item_ids'][:]
        itemlist.extend(temp)
        val_interactions.append([k, v['items'][:], v['items'][-1], itemlist, v['item_ids'][-1], v['item_titles'][:], v['item_titles'][-1], int(v['timestamps'][-1])])


100%|██████████| 1194/1194 [00:00<00:00, 49086.96it/s]


In [197]:
test_interactions = []

for kv in tqdm(test_sessions):
    k = kv[0]
    v = kv[1]
    if len(v['items']) > 10:
        for i in range(min(10, len(v['items']) - 1), len(v['items'])):
            st = max(i - 10, 0)
            test_interactions.append([k, v['items'][st: i], v['items'][i], v['item_ids'][st: i], v['item_ids'][i], v['item_titles'][st: i], v['item_titles'][i], int(v['timestamps'][i])])
    else:
        temp = [pad_item] * (10-len(v['items']))
        itemlist = v['item_ids'][:]
        itemlist.extend(temp)
        test_interactions.append([k, v['items'][:], v['items'][-1], itemlist, v['item_ids'][-1], v['item_titles'][:], v['item_titles'][-1], int(v['timestamps'][-1])])


100%|██████████| 1194/1194 [00:00<00:00, 56391.30it/s]


In [ ]:
Test_interactions = []

for kv in tqdm(test_sessions):
    k = kv[0]
    v = kv[1]

    if len(v['items']) > 10:
        i = len(v['items']) - 1

        st = max(i - 10, 0)
        Test_interactions.append([k, v['items'][st: i], v['items'][i], v['item_ids'][st: i], v['item_ids'][i], v['item_titles'][st: i], v['item_titles'][i], int(v['timestamps'][i])])

    else:
        temp = [pad_item] * (10-len(v['items']))
        itemlist = v['item_ids'][:]
        itemlist.extend(temp)
        Test_interactions.append([k, v['items'][:], v['items'][-1], itemlist, v['item_ids'][-1], v['item_titles'][:], v['item_titles'][-1], int(v['timestamps'][-1])])




In [ ]:
print(train_interactions[0])
print(train_interactions[1])
print(val_interactions[0])
print(val_interactions[1])
print(test_interactions[0])
print(test_interactions[1])
print(Test_interactions[0])
print(Test_interactions[1])

In [ ]:
# writing files
import csv

with open('./train.csv', 'w') as f:
    csvwriter = csv.writer(f)
    csvwriter.writerow(['user_id', 'item_asins', 'item_asin', 'history_item_id', 'item_id', 'history_item_title', 'item_title', 'history_rating', 'rating', 'timestamp'])
    csvwriter.writerows(train_interactions[:])

with open('./val.csv', 'w') as f:
    csvwriter = csv.writer(f)
    csvwriter.writerow(['user_id', 'item_asins', 'item_asin', 'history_item_id', 'item_id', 'history_item_title', 'item_title', 'history_rating', 'rating', 'timestamp'])
    csvwriter.writerows(val_interactions[:])

with open('./test.csv', 'w') as f:
    csvwriter = csv.writer(f)
    csvwriter.writerow(['user_id', 'item_asins', 'item_asin', 'history_item_id', 'item_id', 'history_item_title', 'item_title', 'history_rating', 'rating', 'timestamp'])
    csvwriter.writerows(test_interactions[:])


with open('./Test.csv', 'w') as f:
    csvwriter = csv.writer(f)
    csvwriter.writerow(['user_id', 'item_asins', 'item_asin', 'history_item_id', 'item_id', 'history_item_title', 'item_title', 'history_rating', 'rating', 'timestamp'])
    csvwriter.writerows(Test_interactions[:])

In [ ]:
with open('./id2name.txt', 'w') as f:
    for i in item2id_no_cold.keys():
        f.write(str(item2id_no_cold[i]))
        f.write('::')
        f.write(item2title_no_cold[i])
        f.write('\n')

In [ ]:
def str2list(x):
    x = x[1:-1]
    x = x.split(',')
    x = [int(i) for i in x]
    return x

def str2list2(x):
    x = x[1:-1]
    x = x.split(',')
    return x

In [ ]:
import pandas as pd

train_data_raw = pd.read_csv('./train.csv')

train_data_raw

train_data = train_data_raw[['item_asins', 'history_item_id', 'item_id']]

train_data.columns = ['seq_unpad', 'seq', 'next']

train_data['seq'] = train_data['seq'].apply(lambda x: str2list(x))
train_data['seq_unpad'] = train_data['seq_unpad'].apply(lambda x: str2list2(x))

train_data['next'] = train_data['next'].apply(lambda x: int(x))

train_data['len_seq'] = train_data['seq_unpad'].apply(lambda x: len(x))
min(train_data['len_seq'])

In [ ]:
val_data_raw = pd.read_csv('./val.csv')

val_data_raw

val_data = val_data_raw[['item_asins', 'history_item_id', 'item_id']]

val_data.columns = ['seq_unpad', 'seq', 'next']

val_data['seq'] = val_data['seq'].apply(lambda x: str2list(x))
val_data['seq_unpad'] = val_data['seq_unpad'].apply(lambda x: str2list2(x))

val_data['next'] = val_data['next'].apply(lambda x: int(x))

val_data['len_seq'] = val_data['seq_unpad'].apply(lambda x: len(x))
min(val_data['len_seq'])

In [ ]:
test_data_raw = pd.read_csv('./test.csv')

test_data_raw

test_data = test_data_raw[['item_asins', 'history_item_id', 'item_id']]

test_data.columns = ['seq_unpad', 'seq', 'next']

test_data['seq'] = test_data['seq'].apply(lambda x: str2list(x))
test_data['seq_unpad'] = test_data['seq_unpad'].apply(lambda x: str2list2(x))

test_data['next'] = test_data['next'].apply(lambda x: int(x))

test_data['len_seq'] = test_data['seq_unpad'].apply(lambda x: len(x))
min(test_data['len_seq'])

In [ ]:
Test_data_raw = pd.read_csv('./Test.csv')

Test_data_raw

Test_data = Test_data_raw[['item_asins', 'history_item_id', 'item_id']]

Test_data.columns = ['seq_unpad', 'seq', 'next']

Test_data['seq'] = Test_data['seq'].apply(lambda x: str2list(x))
Test_data['seq_unpad'] = Test_data['seq_unpad'].apply(lambda x: str2list2(x))

Test_data['next'] = Test_data['next'].apply(lambda x: int(x))

Test_data['len_seq'] = Test_data['seq_unpad'].apply(lambda x: len(x))
min(Test_data['len_seq'])

In [ ]:
static_dic = {'seq_size':[10],'item_num':[pad_item]}
data_statis = pd.DataFrame(data=static_dic)

train_data = train_data[['seq', 'next', 'len_seq']]
val_data = val_data[['seq', 'next', 'len_seq']]
test_data = test_data[['seq', 'next', 'len_seq']]
Test_data = Test_data[['seq', 'next', 'len_seq']]

train_data.to_pickle('./train_data.df')
val_data.to_pickle('./val_data.df')
test_data.to_pickle('./ttest_data.df')
Test_data.to_pickle('./Test_data.df')
data_statis.to_pickle('./data_statis.df')

In [ ]:
LL = train_data["seq"].apply(lambda x: len(x))

i = 0

for k in LL:
    if k == 9:
        print(i)

    i += 1